In [3]:
import os
import sys
import subprocess
import shutil
import io

# --- 1. Xoá thư mục chromadb cũ nếu tồn tại ---
if os.path.isdir("chromadb"):
    print("Đang xóa thư mục chromadb cũ...")
    shutil.rmtree("chromadb")

# --- 3. Kiểm tra và cài đặt các thư viện cần thiết ---
def ensure_package(pkg_name, import_name=None):
    try:
        __import__(import_name or pkg_name)
    except ImportError:
        print(f"Thiếu thư viện '{pkg_name}', đang cài đặt...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg_name])
        __import__(import_name or pkg_name)

pkgs = [
    ("chromadb", None),
    ("langchain", None),
    ("ollama", None),
    ("tiktoken", None),
    ("PyPDF2", None)
]

for pkg, imp in pkgs:
    ensure_package(pkg, imp)

Đang xóa thư mục chromadb cũ...


In [ ]:
# --- Đọc PDF với decoder UTF-8 để tránh lỗi tuple ---
import os
from PyPDF2 import PdfReader
from langchain.schema import Document

pdf_path = r"D:\Project_self\pdf_place\CleanCode.pdf"  # Thay đường dẫn tới file PDF của bạn
reader = PdfReader(pdf_path)
docs = []

for i, page in enumerate(reader.pages):
    raw_text = page.extract_text() or ""
    # Nếu raw_text là bytes, decode bằng utf-8 và bỏ ký tự không hợp lệ
    if isinstance(raw_text, (bytes, bytearray)):
        text = raw_text.decode("utf-8", errors="ignore")
    else:
        text = raw_text
    docs.append(Document(
        page_content=text,
        metadata={"source": f"{os.path.basename(pdf_path)}_page_{i+1}"}
    ))

In [ ]:
from PyPDF2 import PdfReader
from langchain.schema import Document
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

In [ ]:
# Sử dụng embedding model local
embed_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
# --- 7. Tạo hoặc tải lại vector database local với fallback khi embed thất bại ---
try:
    vectordb = Chroma.from_documents(
        documents=docs,
        embedding=embed_model,
        persist_directory="chromadb"
    )
    vectordb.persist()
    print("🎉 VectorDB khởi tạo thành công với model embedding hiện tại.")
    
except ValueError as e:
    print(f"[Warning] Embed thất bại: {e}")
    print("Chuyển sang model embedding thay thế: sentence-transformers/all-MiniLM-L6-v2")
    # Cài và dùng MiniLM embedding local thay thế
    from langchain.embeddings import SentenceTransformerEmbeddings
    fallback_embed = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    vectordb = Chroma.from_documents(
        documents=docs,
        embedding=fallback_embed,
        persist_directory="chromadb"
    )
    vectordb.persist()
    embed_model = fallback_embed  # cập nhật cho phần truy vấn sau
    print("🎉 VectorDB đã khởi tạo lại với MiniLM embedding.")

In [1]:
from dotenv import load_dotenv
import os, requests

load_dotenv()  # đọc các biến GEMINI_API_KEY, GEMINI_MODEL,…

API_KEY    = os.getenv("GEMINI_API_KEY")
MODEL_ID   = os.getenv("GEMINI_MODEL") # ví dụ: "gemini-2.5-pro"
PROMPT     = os.getenv(
    "GEMINI_PROMPT",
    "Bạn là một trợ lý hữu ích. Hãy trả lời đầy đủ và chi tiết. Đừng lặp lại câu trả lời nếu không cần thiết."
)
MAX_TOKENS = int(os.getenv("GEMINI_MAX_TOKENS", "256"))
TEMP       = float(os.getenv("GEMINI_TEMPERATURE", "0.7"))

# Kiểm tra
print("API Key   :", "✓" if API_KEY else "✗")
print("Model     :", MODEL_ID)
print("MaxTokens :", MAX_TOKENS)
print("Temperature:", TEMP)



API Key   : ✓
Model     : gemini-2.5-pro
MaxTokens : 1048
Temperature: 0.7


In [18]:
# Cell: Định nghĩa và test với debug
import os
import asyncio
import httpx
import nest_asyncio
from dotenv import load_dotenv
import re
from IPython.display import display, Markdown
from IPython.display import clear_output
# Patch để chạy asyncio.run()/await trong Jupyter
nest_asyncio.apply()

# Load env
load_dotenv()
API_KEY    = os.getenv("GEMINI_API_KEY")
MODEL_ID   = os.getenv("GEMINI_MODEL")       # "gemini-2.5-pro"
MAX_TOKENS = int(os.getenv("GEMINI_MAX_TOKENS", "256"))
TEMP       = float(os.getenv("GEMINI_TEMPERATURE", "0.7"))

BASE_URL = f"https://generativelanguage.googleapis.com/v1/models/{MODEL_ID}:generateContent"

async def generate_content(prompt: str) -> dict:
    payload = {
        "contents": [{"parts": [{"text": prompt}]}],
        "generationConfig": {
            "maxOutputTokens": MAX_TOKENS,
            "temperature": TEMP
        }
    }
    async with httpx.AsyncClient(timeout=30.0) as client:
        resp = await client.post(BASE_URL, params={"key": API_KEY}, json=payload)
        resp.raise_for_status()
        return resp.json()

async def main(prompt: str):
    result = await generate_content(prompt)
    
    # 1) In raw response để debug
    print("🔍 Raw response:")
    import pprint; pprint.pprint(result)
    
    # 2) Show finishReason nếu có
    fr = result.get("candidates", [{}])[0].get("finishReason")
    print(f"\n⚙️ finishReason: {fr}")

    # 3) Extract text nếu có
    cands = result.get("candidates", [])
    if not cands:
        print("❌ No candidates returned.")
        return
    content_obj = cands[0].get("content", {})
    parts = content_obj.get("parts") or []
    # Ghép text và loại bỏ newline ngay tại đây:
    text = "".join(p.get("text", "") for p in parts)
    clean_text = re.sub(r"\s+", " ", text).strip()
    
    if text:
        print("\n🎉 Gemini trả lời đầy đủ:\n")
        print(text)
    else:
        print("\n⚠️ Không tìm thấy text trong parts.")
        print(" Nếu finishReason là MAX_TOKENS, thử tăng GEMINI_MAX_TOKENS lên 512 hoặc 1024 và chạy lại.")
    
    
    display(Markdown(clean_text))
    
    clear_output(wait=True)
    print(clean_text)

In [20]:
# Chạy test
test_prompt = "Hãy giới thiệu về bạn."
asyncio.run(main(test_prompt))


Chào bạn! Tôi là một mô hình ngôn ngữ lớn, được huấn luyện bởi Google. Bạn có thể hình dung tôi như một bộ não kỹ thuật số khổng lồ, được tạo ra để hiểu và xử lý ngôn ngữ của con người. Dưới đây là một vài điểm chính về tôi: **1. Tôi là ai?** * Tôi không phải là một con người, không có cảm xúc, ý thức hay trải nghiệm cá nhân. Tôi là một chương trình máy tính phức tạp. * Mục tiêu chính của tôi là cung cấp thông tin, trả lời câu hỏi và giúp bạn thực hiện các nhiệm vụ liên quan đến ngôn ngữ một cách sáng tạo và hiệu quả. **2. Tôi có thể làm gì?** * **Trả lời câu hỏi:** Tôi có thể trả lời các câu hỏi của bạn về nhiều chủ đề khác nhau, từ khoa học, lịch sử đến văn hóa, nghệ thuật. * **Sáng tạo nội dung:** Tôi có thể giúp bạn viết email, làm thơ, soạn nhạc, viết kịch bản, hoặc thậm chí là tạo ra các ý tưởng mới. * **Tóm tắt và phân tích:** Tôi có thể đọc một văn bản dài và tóm tắt lại những ý chính cho bạn. * **Dịch thuật:** Tôi có thể dịch giữa nhiều ngôn ngữ khác nhau. * **Lập trình:** Tôi

In [ ]:
# Cell 5: Định nghĩa wrapper GeminiLLM dùng call_gemini
from langchain.llms.base import LLM
from pydantic import BaseModel
from typing import Optional, List

class GeminiLLM(LLM, BaseModel):
    model_name: str
    api_key:    str
    max_output_tokens: int = 1000
    temperature:       float = 0.6

    @property
    def _llm_type(self) -> str:
        return "gemini"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        # Gọi hàm call_gemini đã định nghĩa ở Cell 3
        return call_gemini(prompt)

    async def _acall(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return self._call(prompt, stop)

In [ ]:
# Cell 6: Khởi tạo pipeline RAG với GeminiLLM và vectordb
from langchain.chains import RetrievalQA

# 1) Khởi tạo GeminiLLM
gemini_llm = GeminiLLM(
    model_name=MODEL_ID,            # từ Cell 2
    api_key=API_KEY,                # từ Cell 2
    max_output_tokens=MAX_TOKENS,   # từ Cell 2
    temperature=TEMP                # từ Cell 2
)

# 2) Tạo RetrievalQA chain
retriever = vectordb.as_retriever(search_kwargs={"k": 4})
qa_chain = RetrievalQA.from_chain_type(
    llm=gemini_llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# 3) Định nghĩa hàm ask để hỏi và in kết quả + nguồn
def ask(question: str):
    result = qa_chain.invoke({"query": question})
    print("=== ANSWER ===")
    print(result["result"])
    print("\n=== SOURCES ===")
    for doc in result["source_documents"]:
        print(f"- {doc.metadata.get('source', '')}")


In [ ]:
ask("describe details as much as you can about Smell and Heuristic General.")